In [ ]:
import yfinance as yf
import pandas as pd
from google.cloud import bigquery


# Télécharger la liste des tickers du S&P 500
def get_sp500_tickers():
    """Récupère la liste des tickers du S&P 500 via Wikipedia."""
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url, header=0)
    sp500_table = table[0]  # La table principale est la première
    return sp500_table["Symbol"].tolist()

In [ ]:
# Télécharger les données pour tous les tickers
def download_sp500_data(tickers, start_date="2000-01-01", end_date=None):
    """
    Télécharge les données daily de tous les tickers du S&P 500.
    """
    # Obtenir les données via yfinance
    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",  # Organise les données par ticker
        threads=True,  # Téléchargement parallèle
    )
    return data

In [ ]:
if __name__ == "__main__":
    # Étape 1 : Récupérer les tickers
    print("Téléchargement des tickers du S&P 500...")
    sp500_tickers = get_sp500_tickers()
    print(f"Nombre de tickers récupérés : {len(sp500_tickers)}")

    # Étape 2 : Télécharger les données
    print("Téléchargement des données journalières...")
    sp500_data = download_sp500_data(sp500_tickers[0:2], start_date="2020-01-01")

    # Étape 3 : Sauvegarder les données dans un fichier CSV
    print("Sauvegarde des données dans sp500_data.csv...")
    sp500_data.to_csv("sp500_data.csv")
    print("Données sauvegardées avec succès !")

Téléchargement des tickers du S&P 500...
Nombre de tickers récupérés : 502
Téléchargement des données journalières...


[*********************100%***********************]  2 of 2 completed

Sauvegarde des données dans sp500_data.csv...
Données sauvegardées avec succès !


In [ ]:
# Charger les données dans BigQuery
def load_data_to_bigquery(df, table_id, project_id):
    """
    Charge un DataFrame dans une table BigQuery.

    Arguments :
        df : pandas.DataFrame contenant les données à charger.
        table_id : ID complet de la table BigQuery (ex : dataset.table_name).
        project_id : ID du projet Google Cloud.
    """
    client = bigquery.Client(project=project_id)

    # Convertir les données au format BigQuery
    job = client.load_table_from_dataframe(df, table_id)

    # Attendre la fin du job
    job.result()
    print(f"Les données ont été chargées dans {table_id}")

In [ ]:
if __name__ == "__main__":
    # Étape 1 : Récupérer les tickers
    print("Téléchargement des tickers du S&P 500...")
    sp500_tickers = get_sp500_tickers()
    print(f"Nombre de tickers récupérés : {len(sp500_tickers)}")

    # Étape 2 : Télécharger les données
    print("Téléchargement des données journalières...")
    sp500_data = download_sp500_data(sp500_tickers[0:3], start_date="2024-01-01")

    # Restructurer les données pour BigQuery
    print("Restructuration des données pour BigQuery...")
    sp500_data = sp500_data.stack(
        level=0, future_stack=True
    ).reset_index()  # Rendre les tickers une colonne
    sp500_data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]

Téléchargement des tickers du S&P 500...
Nombre de tickers récupérés : 502
Téléchargement des données journalières...


[*********************100%***********************]  3 of 3 completed

Restructuration des données pour BigQuery...


In [ ]:
# Étape 3 : Charger dans BigQuery
print("Chargement des données dans BigQuery...")
PROJECT_ID = "quant-dev-442615"  # Remplacez par votre ID de projet
DATASET_ID = "financial_data"  # Remplacez par le nom de votre dataset
TABLE_ID = "sp500_data"  # Nom de la table

# Charger les données
load_data_to_bigquery(
    sp500_data,
    table_id=f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}",
    project_id=PROJECT_ID,
)

Chargement des données dans BigQuery...


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [57]:
from datetime import datetime, timedelta


# Télécharger les données du jour précédent
def download_previous_day_data(tickers):
    """
    Télécharge les données daily pour les tickers du jour précédent.
    """
    yesterday = datetime.now() - timedelta(days=1)
    start_date = yesterday.strftime("%Y-%m-%d")
    end_date = start_date  # Une seule journée
    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",
        threads=True,
    )
    # Restructurer les données
    data = data.stack(level=0, future_stack=True).reset_index()
    data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]
    return data

In [ ]:
tickers = get_sp500_tickers()

# Étape 2 : Télécharger les données
print("Téléchargement des données du jour précédent...")
sp500_data = download_previous_day_data(tickers[0:2])
sp500_data["Date"] = pd.to_datetime(sp500_data["Date"]).dt.date

[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['AOS', 'MMM']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-11-24 -> 2024-11-24)')


Téléchargement des données du jour précédent...


In [59]:
sp500_data

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume


In [ ]:
tickers = ["AAPL", "MSFT"]
yesterday = datetime.now() - timedelta(days=1)
start_date = yesterday.strftime("%Y-%m-%d")
end_date = start_date  # Une seule journée
start_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
end_date = (datetime.now() - timedelta(days=0)).strftime("%Y-%m-%d")

print(start_date)
print(end_date)
data = yf.download(
    tickers=tickers,
    start=start_date,
    end=end_date,
    interval="1d",
    group_by="ticker",
    threads=True,
)
# Restructurer les données
data = data.stack(level=0, future_stack=True).reset_index()
data.columns = [
    "Date",
    "Ticker",
    "Open",
    "High",
    "Low",
    "Close",
    "Adj Close",
    "Volume",
]
print(data)

[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['MSFT', 'AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-11-24 -> 2024-11-25)')


2024-11-24
2024-11-25
Empty DataFrame
Columns: [Date, Ticker, Open, High, Low, Close, Adj Close, Volume]
Index: []
